In [1]:
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, AutoConfig
import os
from datasets import load_from_disk
import random
import json
from tqdm import tqdm
from score_preds import get_score

cache_dir = '/scratch/workspace/wenlongzhao_umass_edu-analyze/Bidirectional-Teacher-Student-Communication-for-LLM-Alignment/transformers_cache'
os.environ['TRANSFORMERS_CACHE'] = cache_dir



In [2]:
# Loading data
data_eval_path = "../datasets/gsm8k/test/"
data_test = load_from_disk(data_eval_path)
data_train = load_from_disk("../datasets/gsm8k/train/")

# Loading model
hf_token = os.getenv("hf_token")
model_name = "meta-llama/Llama-3.2-3B-Instruct"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token, cache_dir=cache_dir)
# Defaults to float32
model = AutoModelForCausalLM.from_pretrained(model_name, token=hf_token, cache_dir=cache_dir, device_map=device, torch_dtype=torch.bfloat16)
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = 'left'


# Set seed for reproducibility
seed = 42
torch.manual_seed(seed)




Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
model.device

device(type='cuda', index=0)

In [4]:
# 8-shot prompt from : https://huggingface.co/datasets/meta-llama/Llama-3.2-1B-Instruct-evals/viewer/Llama-3.2-1B-Instruct-evals__gsm8k__details?row=0
prompt = "<|start_header_id|>user<|end_header_id|>\n\nGiven the following problem, reason and give a final answer to the problem.\nProblem: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?\nYour response should end with \'The final answer is [answer]\' where [answer] is the response to the problem.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThere are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. The final answer is 6<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven the following problem, reason and give a final answer to the problem.\nProblem: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?\nYour response should end with \'The final answer is [answer]\' where [answer] is the response to the problem.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThere are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. The final answer is 5<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven the following problem, reason and give a final answer to the problem.\nProblem: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?\nYour response should end with \'The final answer is [answer]\' where [answer] is the response to the problem.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nOriginally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39. The final answer is 39<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven the following problem, reason and give a final answer to the problem.\nProblem: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?\nYour response should end with \'The final answer is [answer]\' where [answer] is the response to the problem.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nJason started with 20 lollipops. Then he had 12 after giving some to Denny. So he gave Denny 20 - 12 = 8. The final answer is 8<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven the following problem, reason and give a final answer to the problem.\nProblem: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?\nYour response should end with \'The final answer is [answer]\' where [answer] is the response to the problem.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nShawn started with 5 toys. If he got 2 toys each from his mom and dad, then that is 4 more toys. 5 + 4 = 9. The final answer is 9<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven the following problem, reason and give a final answer to the problem.\nProblem: There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room?\nYour response should end with \'The final answer is [answer]\' where [answer] is the response to the problem.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThere were originally 9 computers. For each of 4 days, 5 more computers were added. So 5 * 4 = 20 computers were added. 9 + 20 is 29. The final answer is 29<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven the following problem, reason and give a final answer to the problem.\nProblem: Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?\nYour response should end with \'The final answer is [answer]\' where [answer] is the response to the problem.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nMichael started with 58 golf balls. After losing 23 on tuesday, he had 58 - 23 = 35. After losing 2 more, he had 35 - 2 = 33 golf balls. The final answer is 33<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven the following problem, reason and give a final answer to the problem.\nProblem: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?\nYour response should end with \'The final answer is [answer]\' where [answer] is the response to the problem.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nOlivia had 23 dollars. 5 bagels for 3 dollars each will be 5 x 3 = 15 dollars. So she has 23 - 15 dollars left. 23 - 15 is 8. The final answer is 8<|eot_id|>"


model.eval()
generated_outputs=[]
# Adjust batch size according to your GPU memory capacity
# With vram=80G, 1B - 64, 3B - 64, 8B - 32
batch_size=64
temp = 0.0
top_p = 0
top_k = 0

stop_strings = ['<|eot_id|>', '<|start_header_id|>user<|end_header_id|>','Q:', '</s>','<|im_end|>']
regex_pattern = r"The final answer is ((-?[$0-9.,]{2,})|(-?[0-9]+))"



In [5]:
import re
for i in tqdm(range(0, len(data_test["question"]), batch_size), desc="Processing questions"):
    batch_questions = data_test["question"][i:i+batch_size]
    inputs = []
    for q in batch_questions:

        inputs.append(prompt + f'<|start_header_id|>user<|end_header_id|>\n\nGiven the following problem, reason and give a final answer to the problem.\nProblem: {q}\nYour response should end with \'The final answer is [answer]\' where [answer] is the response to the problem.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n') 

    tokenized_inputs = tokenizer(inputs, return_tensors="pt", padding=True, truncation=True)
    tokenized_inputs.to(device)

    with torch.no_grad():
        output = model.generate(**tokenized_inputs, max_new_tokens = 1024,pad_token_id=tokenizer.pad_token_id,do_sample=False, stop_strings = stop_strings, tokenizer = tokenizer )
      
    for j, o in enumerate(output):
        generated_text = tokenizer.decode(o, skip_special_tokens=True)
        answer = generated_text.split("Your response should end with \'The final answer is [answer]\' where [answer] is the response to the problem.assistant")[-1]
        generated_outputs.append({"input": inputs[j], "output": generated_text, "question": batch_questions[j], "answer":answer})
        


Processing questions:   0%|          | 0/21 [00:00<?, ?it/s]/home/kchimmad_umass_edu/.conda/envs/IESLAnalyse/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/kchimmad_umass_edu/.conda/envs/IESLAnalyse/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Processing questions: 100%|██████████| 21/21 [22:48<00:00, 65.17s/it] 


In [6]:
output_file_name = f"../outputs/gsm8k/LLaMA3B/generated_outputs_test_with_regex_and_stop_words_1.json" 
with open(output_file_name, "w") as f:
    json.dump(generated_outputs, f, indent=4)



In [7]:
score = get_score(data_eval_path,output_file_name)
print("SCORE of LLaMA 3B Model is: ",score)

SCORE of LLaMA 3B Model is:  0.7407126611068992
